# Digit Recognizer using multi-layer CNN
* https://www.kaggle.com/c/digit-recognizer

In [1]:
import numpy as np
import pandas as pd

## prepareing data
Download data from https://www.kaggle.com/c/digit-recognizer/data

In [2]:
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

validation_size = 2000

In [3]:
#DATA_DIR = "/home/carnd/dataset/"
DATASRC_DIR = "/home/tsu-nera/repo/kaggle/input/"
train = pd.read_csv(DATASRC_DIR+'digit-recognizer/train.csv')
test  = pd.read_csv(DATASRC_DIR+'digit-recognizer/test.csv')

In [4]:
(train.shape, test.shape)

((42000, 785), (28000, 784))

split train data to labels and pixels.

In [5]:
features = (train.ix[:,1:].values).astype('float32')
labels = pd.get_dummies(train.ix[:,0]).astype('float32')
(features.shape, labels.shape)

/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


((42000, 784), (42000, 10))

In [6]:
# split data into training & validation
valid_features = features[:validation_size]
valid_labels = labels[:validation_size]

train_features = features[validation_size:]
train_labels = labels[validation_size:]
(train_features.shape, train_labels.shape, valid_features.shape, valid_labels.shape)

((40000, 784), (40000, 10), (2000, 784), (2000, 10))

In [7]:
test_features = (test.values).astype('float32')
(test_features.shape)

(28000, 784)

In [8]:
features = features.reshape(-1,28,28)
valid_features = valid_features.reshape(-1,28,28)
test_features = test_features.reshape(-1, 28, 28)

(features.shape, valid_features.shape, test_features.shape)

((42000, 28, 28), (2000, 28, 28), (28000, 28, 28))

In [9]:
features = np.expand_dims(features,3)
valid_features = np.expand_dims(valid_features,3)
test_features = np.expand_dims(test_features,3)
(features.shape, valid_features.shape, test_features.shape)

((42000, 28, 28, 1), (2000, 28, 28, 1), (28000, 28, 28, 1))

In [10]:
mean_px = features.mean().astype(np.float32)
std_px = features.std().astype(np.float32)

In [11]:
def norm_input(x): return (x-mean_px)/std_px

## Make a TensorFlow Graph

In [12]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Lambda, Conv2D, BatchNormalization, MaxPool2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [ ]:
nf = 64

In [ ]:
def get_model():
    return [
    Lambda(norm_input, input_shape=(28,28,1)),
    Conv2D(nf, kernel_size=(3,3), activation='relu'),
    BatchNormalization(axis=1),
    Conv2D(nf, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    BatchNormalization(axis=1),
    Conv2D(nf, kernel_size=(3,3), activation='relu'),
    BatchNormalization(axis=1),
    Conv2D(nf, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
    ]

In [ ]:
model = Sequential(get_model())

In [ ]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#model.summary()

## Training

In [14]:
batch_size = 64

In [15]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                            height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(features, labels, batch_size=batch_size)
test_batches = gen.flow(valid_features, valid_labels, batch_size=batch_size)

In [16]:
model.fit_generator(batches, batches.n//batch_size, epochs=1,
                    validation_data=test_batches, validation_steps=test_batches.n//batch_size)

NameError: name 'model' is not defined

##  Tuning with hyparas
* [maxpumperla/hyperas: Keras + Hyperopt: A very simple wrapper for convenient hyperparameter optimization](https://github.com/maxpumperla/hyperas)

バグかあるみたいでエラーする。動かないよ。
* [TypeError: sequence item 1: expected string, NoneType found · Issue #85 · maxpumperla/hyperas](https://github.com/maxpumperla/hyperas/issues/85)

バグってるけどクローズされてしまったみたい。
* [Hyperas in Keras (Older Version) · Issue #27 · maxpumperla/hyperas](https://github.com/maxpumperla/hyperas/issues/27)

In [76]:
from hyperas import optim

from keras.optimizers import Nadam, Adam, RMSprop
from keras.layers.advanced_activations import PReLU

In [74]:
def get_model2():
    model = Sequential()
    model.add(Lambda(norm_input, input_shape=(28,28,1)))
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D())
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D())
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(10, activation='softmax'))
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [75]:
model = get_model2()

TypeError: unsupported operand type(s) for +: 'int' and 'set'

## Testing

In [ ]:
classes = model.predict_classes(test_features)

In [ ]:
classes[:5]

## Write to file

In [ ]:
submissions = pd.DataFrame({"ImageId": list(range(1, len(classes)+1)),
                             "Label": classes})
submissions.to_csv("output.csv", index=False, header=True)